In [1]:
import warnings
warnings.simplefilter('ignore')

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import auc, roc_curve, classification_report, precision_score, recall_score, accuracy_score, roc_auc_score 

 
from sklearn.preprocessing import StandardScaler

In [2]:


import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))



/kaggle/input/titanic/train.csv
/kaggle/input/titanic/gender_submission.csv
/kaggle/input/titanic/test.csv


In [3]:
#traindata=pd.read_csv(r'/home/jovyan/work/loan_table.csv', parse_dates=['date'])
#parse_dates是将所有字符串形式的日期转换为日期形式

#TitanicPD = pd.read_csv(r'/kaggle/input/train.csv') 
traindata = pd.read_csv('/kaggle/input/titanic/train.csv') 
traindata.head()
traindata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [4]:
testdata = pd.read_csv('/kaggle/input/titanic/test.csv') 
testdata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   Pclass       418 non-null    int64  
 2   Name         418 non-null    object 
 3   Sex          418 non-null    object 
 4   Age          332 non-null    float64
 5   SibSp        418 non-null    int64  
 6   Parch        418 non-null    int64  
 7   Ticket       418 non-null    object 
 8   Fare         417 non-null    float64
 9   Cabin        91 non-null     object 
 10  Embarked     418 non-null    object 
dtypes: float64(2), int64(4), object(5)
memory usage: 36.0+ KB


In [5]:
traindata['Age'] = traindata['Age'].fillna(traindata['Age'].median())
testdata['Age'] = testdata['Age'].fillna(testdata['Age'].median())
traindata['Cabin'] = traindata['Cabin'].fillna('U')
testdata['Cabin'] = testdata['Cabin'].fillna('U')
traindata.info()
testdata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          891 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        891 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   Pclass  

In [6]:
traindata['Embarked'].value_counts()

S    644
C    168
Q     77
Name: Embarked, dtype: int64

In [7]:
traindata['Embarked'] = traindata['Embarked'].fillna('S')
testdata['Fare'] = testdata['Fare'].fillna(testdata['Fare'].median())

In [8]:
traindata.info()
testdata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          891 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        891 non-null    object 
 11  Embarked     891 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   Pclass  

In [9]:
def get_title(name):
    title = name.split('.')[0].split(',')[-1].strip()
    return title
traindata['Title'] = traindata['Name'].map(get_title)
testdata['Title'] = testdata['Name'].map(get_title)

In [10]:
encoder = LabelEncoder()
traindata['Title'] = encoder.fit_transform(traindata['Title'])
encoder = LabelEncoder()
traindata['Sex'] = encoder.fit_transform(traindata['Sex'])
encoder = LabelEncoder()
traindata['Embarked'] = encoder.fit_transform(traindata['Embarked'])
traindata['Cabin'] = traindata['Cabin'].map(lambda x: 0 if x == 'U' else 1)

encoder = LabelEncoder()
testdata['Title'] = encoder.fit_transform(testdata['Title'])
encoder = LabelEncoder()
testdata['Sex'] = encoder.fit_transform(testdata['Sex'])
encoder = LabelEncoder()
testdata['Embarked'] = encoder.fit_transform(testdata['Embarked'])
testdata['Cabin'] = testdata['Cabin'].map(lambda x: 0 if x == 'U' else 1)

In [11]:
traindata.corr()['Survived'].sort_values(ascending = False)

Survived       1.000000
Cabin          0.316912
Fare           0.257307
Parch          0.081629
PassengerId   -0.005007
SibSp         -0.035322
Age           -0.064910
Embarked      -0.167675
Title         -0.193635
Pclass        -0.338481
Sex           -0.543351
Name: Survived, dtype: float64

In [12]:
Xtrain = traindata[['Pclass', 'Parch','Sex','Fare','Cabin','Embarked']]

ytrain = traindata['Survived']
Xtest= testdata[['Pclass', 'Parch','Sex','Fare','Cabin','Embarked']]


In [13]:
import sklearn.linear_model as sk
    
ss = StandardScaler()
X_train = ss.fit_transform(Xtrain)
X_test = ss.transform(Xtest)

    
LR=sk.LogisticRegressionCV(cv=5, random_state=0)
lrmodel=LR.fit(X_train,ytrain)

lrpred=lrmodel.predict(X_test)  #predicted laber on test dataset
lrpred_prob=lrmodel.predict_proba(X_test)[:,1]  #predicted probability on test dataset
#print(lrmodel.coef_)

#lrpred=lrmodel.predict(X_test)  #predicted laber on test dataset
#lrpred_prob=lrmodel.predict_proba(X_test)[:,1]  #predicted probability on test dataset
#print(lrmodel.coef_)

#print(accuracy_score(y_test,lrpred))
#print(precision_score(y_test,lrpred))
lrpred=lrmodel.predict(X_train)  #predicted laber on test dataset

print(accuracy_score(ytrain,lrpred))
print(precision_score(ytrain,lrpred))

0.7901234567901234
0.7272727272727273


In [14]:

import xgboost as xgb
from xgboost.sklearn import XGBClassifier
from xgboost import plot_tree
from xgboost import plot_importance
ss = StandardScaler()
X_train = ss.fit_transform()
 

train_matrix = xgb.DMatrix(X_train,ytrain)

params = {'booster':'gbtree','objective': 'binary:logistic','colsample_bytree': 0.8,'learning_rate': 0.1,
                'max_depth': 5, 'alpha': 10}
cv_results = xgb.cv(dtrain=train_matrix, params=params, nfold=3,
                    num_boost_round=50,early_stopping_rounds=30,metrics='auc', as_pandas=True, seed=123)

n_best_trees = cv_results.shape[0]
print("best number of trees: {}".format(n_best_trees))  # output 53

watchlist = [(train_matrix, 'train')]
gbt = xgb.train(params, train_matrix, n_best_trees,watchlist)  #打印错误率，准确率=1-错误率


TypeError: fit_transform() missing 1 required positional argument: 'X'

In [15]:

dtest = xgb.DMatrix(Xtrain)
ans = gbt.predict(dtest)

predictions = [round(i) for i in ans]
accuracy = accuracy_score(ytrain, predictions)
accuracy 

NameError: name 'gbt' is not defined

In [16]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/gender_submission.csv
/kaggle/input/titanic/test.csv
